In [1]:
# init prerequisites
%reload_ext autoreload
%autoreload 2
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os, re, math, time, random
from pprint import pprint

import util
from util import print_warn, horline

print("OpenCV {}".format(cv.__version__))
print("Numpy {}".format(np.__version__))

print_warn("Warning Text!")
horline()

OpenCV 3.4.1
Numpy 1.16.2
 Warning Text! 
________________________________________________________________________________________________________________________


In [2]:
def init_camera():
    global cap
    cap = cv.VideoCapture(0)
    print("CAP_PROP_FRAME_WIDTH:", cap.get(cv.CAP_PROP_FRAME_WIDTH))
    print("CAP_PROP_FRAME_HEIGHT:", cap.get(cv.CAP_PROP_FRAME_HEIGHT))
    cap.set(cv.CAP_PROP_FRAME_WIDTH, 1920)
    cap.set(cv.CAP_PROP_FRAME_HEIGHT, 1080)
    print("CAP_PROP_FRAME_WIDTH:", cap.get(cv.CAP_PROP_FRAME_WIDTH))
    print("CAP_PROP_FRAME_HEIGHT:", cap.get(cv.CAP_PROP_FRAME_HEIGHT))

In [ ]:
# realtime video
cv.destroyAllWindows()
cv.namedWindow('test', cv.WINDOW_NORMAL)
cv.resizeWindow('test', 640, 480)

try:
    init_camera()
    while(True):
        ret, frame = cap.read()
        assert ret, "No image read from camera!"
        frame = cv.resize(frame, (int(frame.shape[1]*0.125), int(frame.shape[0]*0.125)))
        cv.imshow('test',frame)
        k = cv.waitKey(10) & 0xff
        if k == 27:
            break
        
finally:
    cap.release()
    cv.destroyAllWindows()

In [3]:
# realtime video with dense farnebäck flow
cv.destroyAllWindows()

# initialization
clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
cv.destroyAllWindows()
#cv.namedWindow('frame', cv.WINDOW_NORMAL)
#cv.resizeWindow('frame', 600, 600)
#cv.namedWindow('bgr', cv.WINDOW_NORMAL)
#cv.resizeWindow('bgr', 600, 600)
#cv.namedWindow('mag', cv.WINDOW_NORMAL)
#cv.resizeWindow('mag', 600, 600)
#cv.namedWindow('ang', cv.WINDOW_NORMAL)
#cv.resizeWindow('ang', 600, 600)

init_camera()

# read input function
def get_next(index=None):
    ret, frame = cap.read()
    assert ret, "No image read from camera!"
    col = cv.resize(frame, (int(frame.shape[1]*0.125), int(frame.shape[0]*0.125)))
    #col = cv.medianBlur(col, 31)
    gray = cv.cvtColor(col, cv.COLOR_BGR2GRAY)
    #gray = clahe.apply(gray)
    return col, gray

outputs = dict()
def output(file, image):
    if len(image.shape) == 2:
        image = cv.cvtColor(image, cv.COLOR_GRAY2BGR)
    if file not in outputs:
        outh,outw,_ = image.shape
        print(image.shape)
        videopath = file
        print("saving video to {}".format(videopath))
        outputs[file] = cv.VideoWriter(videopath, cv.VideoWriter_fourcc(*'MJPG'), 30, (outw,outh))    
    outputs[file].write(image)

def close_output():
    for file in outputs:
        outputs[file].release()

# read first image
prev, prev_gray = get_next(0)

# prepare output
hsv = np.zeros_like(prev)
hsv[:,:,1] = 255

while(True):
    # read next input image
    next, next_gray = get_next()
    
    # calculate optical flow
    flow = cv.calcOpticalFlowFarneback(prev_gray,next_gray, None, 0.5, 6, 15, 3, 5, 1.2, 0)
    mag, ang = cv.cartToPolar(flow[:,:,0], flow[:,:,1])
    hsv[:,:,0] = ang*180/np.pi/2
    #hsv[:,:,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
    _, tresh = cv.threshold(mag,8,255,cv.THRESH_TRUNC)
    hsv[:,:,2] = tresh * 31    
    bgr = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
    #cv.imshow('mag',hsv[:,:,2])
    #cv.imshow('ang',hsv[:,:,0])
    #cv.imshow('bgr',bgr)
    #cv.imshow('frame',next_gray)
    
    l1 = np.hstack((cv.cvtColor(hsv[:,:,2], cv.COLOR_GRAY2BGR), cv.cvtColor(hsv[:,:,0], cv.COLOR_GRAY2BGR)))
    l2 = np.hstack((bgr, cv.cvtColor(next_gray, cv.COLOR_GRAY2BGR)))
    stacked = np.vstack((l1, l2))
    cv.imshow('output',stacked)
    output("output/realtime.avi", stacked)
    
    
    k = cv.waitKey(1) & 0xff
    if k == 27:
        break
        
    # update the previous frame
    prev = next
    prev_gray = next_gray
        
close_output()
cv.destroyAllWindows()

CAP_PROP_FRAME_WIDTH: 640.0
CAP_PROP_FRAME_HEIGHT: 480.0
CAP_PROP_FRAME_WIDTH: 1920.0
CAP_PROP_FRAME_HEIGHT: 1080.0
(270, 480, 3)
saving video to output/realtime.avi
